In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.postprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
V = 16
model = 'deberta-v3.len1280.flag-encode_targets-cls-crank'
model2 = 'deberta-v3.len1280.flag-encode_targets-cls'

In [3]:
oof = gezi.read_oof(model, V, return_dict=False)

In [4]:
oof

,text_id,label,n_words,pred,fold
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.795, 2.559, 2.848, 2.691, 2.424, 2.572]",0
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.094, 3.012, 3.191, 3.129, 3.229, 3.207]",0
2,01501F95B8B2,"[3.0, 4.0, 3.0, 4.0, 3.5, 3.0]",676,"[3.596, 3.592, 3.66, 3.803, 3.648, 3.465]",0
3,017D2125A1DE,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",379,"[2.266, 2.174, 2.668, 2.326, 2.213, 2.191]",0
4,0230A077F5B6,"[3.0, 2.5, 2.5, 2.5, 2.0, 2.5]",536,"[2.842, 2.494, 2.818, 2.623, 2.377, 2.207]",0
...,...,...,...,...,...
3906,FF7FE6BD8509,"[3.5, 3.5, 3.5, 3.5, 3.5, 3.5]",622,"[3.463, 3.3, 3.482, 3.564, 3.557, 3.252]",4
3907,FFA6690BC309,"[4.0, 3.5, 4.0, 3.5, 3.5, 4.0]",624,"[3.596, 3.143, 3.375, 3.322, 3.107, 3.383]",4
3908,FFAEAF8D0C90,"[2.5, 2.0, 2.5, 1.5, 2.0, 2.0]",496,"[2.645, 2.3, 2.633, 2.31, 2.14, 2.111]",4
3909,FFCDB2524616,"[2.5, 3.0, 3.0, 4.0, 3.5, 3.0]",263,"[3.291, 3.344, 3.463, 3.586, 3.594, 3.4]",4


In [5]:
def convert_oof(oof, index=0):
  res = {}
  res[f'mean_model{index}'] = []
  res[f'argmax{index}'] = []
  for target in TARGETS:
    res[target] = []
    res[f'{target}_model{index}'] = []
    res[f'{target}_abs_delta_model{index}'] = []
    
  for row in oof.itertuples():
    row = row._asdict()
    res[f'mean_model{index}'].append(row['pred'].mean())
    res[f'argmax{index}'].append(np.argmax(abs(row[f'pred'] - row['pred'].mean())))
    for i, target in enumerate(TARGETS):
      res[target].append(row['label'][i])
      res[f'{target}_model{index}'].append(row[f'pred'][i])
      res[f'{target}_abs_delta_model{index}'].append(abs(row[f'pred'][i] - row['pred'].mean()) / row['pred'].mean())
  for key in res:
    oof[key] = res[key]
  return oof

In [6]:
excl_cols = ['label', 'pred', 'fold', 'n_words'] + TARGETS

In [7]:
oof = convert_oof(oof)
oof.head(2)

,text_id,label,n_words,pred,fold,mean_model0,argmax0,cohesion,cohesion_model0,cohesion_abs_delta_model0,...,vocabulary_abs_delta_model0,phraseology,phraseology_model0,phraseology_abs_delta_model0,grammar,grammar_model0,grammar_abs_delta_model0,conventions,conventions_model0,conventions_abs_delta_model0
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.795, 2.559, 2.848, 2.691, 2.424, 2.572]",0,2.648438,4,2.5,2.794922,0.055298,...,0.075195,2.0,2.691406,0.016220,2.0,2.423828,0.084778,2.5,2.572266,0.028763
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.094, 3.012, 3.191, 3.129, 3.229, 3.207]",0,3.144531,1,3.5,3.093750,0.016144,...,0.014908,3.5,3.128906,0.004971,3.5,3.228516,0.026703,4.0,3.207031,0.019882


In [8]:
oof2 = gezi.read_oof(model2, V, return_dict=False)

In [9]:
oof2 = convert_oof(oof2, index=1)

In [10]:
oof = oof.merge(oof2[[col for col in oof2.columns if col not in excl_cols]], on='text_id')
oof

,text_id,label,n_words,pred,fold,mean_model0,argmax0,cohesion,cohesion_model0,cohesion_abs_delta_model0,...,syntax_model1,syntax_abs_delta_model1,vocabulary_model1,vocabulary_abs_delta_model1,phraseology_model1,phraseology_abs_delta_model1,grammar_model1,grammar_abs_delta_model1,conventions_model1,conventions_abs_delta_model1
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.795, 2.559, 2.848, 2.691, 2.424, 2.572]",0,2.648438,4,2.5,2.794922,0.055298,...,2.621094,0.020432,2.851562,0.065674,2.642578,0.012405,2.492188,0.068604,2.634766,0.015327
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.094, 3.012, 3.191, 3.129, 3.229, 3.207]",0,3.144531,1,3.5,3.093750,0.016144,...,3.082031,0.020477,3.193359,0.014900,3.164062,0.005585,3.113281,0.010551,3.230469,0.026688
2,01501F95B8B2,"[3.0, 4.0, 3.0, 4.0, 3.5, 3.0]",676,"[3.596, 3.592, 3.66, 3.803, 3.648, 3.465]",0,3.626953,3,3.0,3.595703,0.008614,...,3.619141,0.015930,3.740234,0.016998,3.808594,0.035583,3.748047,0.019119,3.523438,0.041962
3,017D2125A1DE,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",379,"[2.266, 2.174, 2.668, 2.326, 2.213, 2.191]",0,2.306641,2,2.5,2.265625,0.017776,...,2.373047,0.028778,2.744141,0.123108,2.365234,0.031982,2.310547,0.054352,2.408203,0.014389
4,0230A077F5B6,"[3.0, 2.5, 2.5, 2.5, 2.0, 2.5]",536,"[2.842, 2.494, 2.818, 2.623, 2.377, 2.207]",0,2.560547,5,3.0,2.841797,0.109863,...,2.546875,0.028320,2.851562,0.087952,2.623047,0.000745,2.466797,0.058868,2.353516,0.102112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3906,FF7FE6BD8509,"[3.5, 3.5, 3.5, 3.5, 3.5, 3.5]",622,"[3.463, 3.3, 3.482, 3.564, 3.557, 3.252]",4,3.437500,5,3.5,3.462891,0.007385,...,3.369141,0.028717,3.548828,0.023087,3.583984,0.033234,3.521484,0.015205,3.328125,0.040527
3907,FFA6690BC309,"[4.0, 3.5, 4.0, 3.5, 3.5, 4.0]",624,"[3.596, 3.143, 3.375, 3.322, 3.107, 3.383]",4,3.320312,0,4.0,3.595703,0.082947,...,3.138672,0.046844,3.386719,0.028473,3.330078,0.011269,3.150391,0.043304,3.353516,0.018387
3908,FFAEAF8D0C90,"[2.5, 2.0, 2.5, 1.5, 2.0, 2.0]",496,"[2.645, 2.3, 2.633, 2.31, 2.14, 2.111]",4,2.357422,0,2.5,2.644531,0.121765,...,2.324219,0.009155,2.578125,0.099060,2.308594,0.015823,2.146484,0.084900,2.154297,0.081604
3909,FFCDB2524616,"[2.5, 3.0, 3.0, 4.0, 3.5, 3.0]",263,"[3.291, 3.344, 3.463, 3.586, 3.594, 3.4]",4,3.445312,0,2.5,3.291016,0.044800,...,3.376953,0.027557,3.455078,0.005062,3.550781,0.022491,3.681641,0.060181,3.408203,0.018555


In [11]:
[col for col in oof.columns if col not in excl_cols]

['text_id',
 'mean_model0',
 'argmax0',
 'cohesion_model0',
 'cohesion_abs_delta_model0',
 'syntax_model0',
 'syntax_abs_delta_model0',
 'vocabulary_model0',
 'vocabulary_abs_delta_model0',
 'phraseology_model0',
 'phraseology_abs_delta_model0',
 'grammar_model0',
 'grammar_abs_delta_model0',
 'conventions_model0',
 'conventions_abs_delta_model0',
 'mean_model1',
 'argmax1',
 'cohesion_model1',
 'cohesion_abs_delta_model1',
 'syntax_model1',
 'syntax_abs_delta_model1',
 'vocabulary_model1',
 'vocabulary_abs_delta_model1',
 'phraseology_model1',
 'phraseology_abs_delta_model1',
 'grammar_model1',
 'grammar_abs_delta_model1',
 'conventions_model1',
 'conventions_abs_delta_model1']

In [12]:
fold = 0
dvalid = oof[oof.fold==fold]
dtrain = oof[oof.fold!=fold]

In [13]:
dvalid

,text_id,label,n_words,pred,fold,mean_model0,argmax0,cohesion,cohesion_model0,cohesion_abs_delta_model0,...,syntax_model1,syntax_abs_delta_model1,vocabulary_model1,vocabulary_abs_delta_model1,phraseology_model1,phraseology_abs_delta_model1,grammar_model1,grammar_abs_delta_model1,conventions_model1,conventions_abs_delta_model1
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.795, 2.559, 2.848, 2.691, 2.424, 2.572]",0,2.648438,4,2.5,2.794922,0.055298,...,2.621094,0.020432,2.851562,0.065674,2.642578,0.012405,2.492188,0.068604,2.634766,0.015327
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.094, 3.012, 3.191, 3.129, 3.229, 3.207]",0,3.144531,1,3.5,3.093750,0.016144,...,3.082031,0.020477,3.193359,0.014900,3.164062,0.005585,3.113281,0.010551,3.230469,0.026688
2,01501F95B8B2,"[3.0, 4.0, 3.0, 4.0, 3.5, 3.0]",676,"[3.596, 3.592, 3.66, 3.803, 3.648, 3.465]",0,3.626953,3,3.0,3.595703,0.008614,...,3.619141,0.015930,3.740234,0.016998,3.808594,0.035583,3.748047,0.019119,3.523438,0.041962
3,017D2125A1DE,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",379,"[2.266, 2.174, 2.668, 2.326, 2.213, 2.191]",0,2.306641,2,2.5,2.265625,0.017776,...,2.373047,0.028778,2.744141,0.123108,2.365234,0.031982,2.310547,0.054352,2.408203,0.014389
4,0230A077F5B6,"[3.0, 2.5, 2.5, 2.5, 2.0, 2.5]",536,"[2.842, 2.494, 2.818, 2.623, 2.377, 2.207]",0,2.560547,5,3.0,2.841797,0.109863,...,2.546875,0.028320,2.851562,0.087952,2.623047,0.000745,2.466797,0.058868,2.353516,0.102112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,FF2B30052436,"[2.0, 2.5, 3.0, 2.5, 3.0, 3.0]",508,"[2.88, 2.77, 3.21, 3.037, 2.9, 2.959]",0,2.958984,2,2.0,2.880859,0.026398,...,2.824219,0.043640,3.113281,0.054230,3.023438,0.023804,2.919922,0.011246,2.966797,0.004631
778,FF76D397A73C,"[3.5, 3.0, 3.0, 3.5, 2.5, 2.5]",389,"[2.979, 2.686, 3.055, 2.875, 2.713, 2.572]",0,2.812500,2,3.5,2.978516,0.059021,...,2.718750,0.029968,3.023438,0.078735,2.794922,0.002787,2.759766,0.015327,2.648438,0.055054
779,FF8BAC351714,"[2.5, 2.0, 3.0, 3.0, 2.0, 2.5]",338,"[2.656, 2.348, 2.719, 2.525, 2.418, 2.283]",0,2.492188,2,2.5,2.656250,0.065857,...,2.392578,0.041473,2.738281,0.097046,2.544922,0.019562,2.445312,0.020340,2.285156,0.084534
780,FF9469424ED0,"[3.5, 3.5, 4.5, 3.5, 4.0, 4.0]",642,"[3.932, 4.066, 4.24, 4.18, 4.324, 4.25]",0,4.164062,0,3.5,3.931641,0.055817,...,4.132812,0.012138,4.226562,0.010269,4.179688,0.000934,4.304688,0.028946,4.304688,0.028946


In [14]:
cols = [col for col in oof.columns if col not in excl_cols + ['text_id']]
# cols += [
#   'n_words'
# ]
ic(cols)
cols

[11/30/22 13:25:41] 2569886194.py:5 in <module>
                    cols: ['mean_model0',
                           'argmax0',
                           'cohesion_model0',
                           'cohesion_abs_delta_model0',
                           'syntax_model0',
                           'syntax_abs_delta_model0',
                           'vocabulary_model0',
                           'vocabulary_abs_delta_model0',
                           'phraseology_model0',
                           'phraseology_abs_delta_model0',
                           'grammar_model0',
                           'grammar_abs_delta_model0',
                           'conventions_model0',
                           'conventions_abs_delta_model0',
                           'mean_model1',
                           'argmax1',
                           'cohesion_model1',
                           'cohesion_abs_delta_model1',
                           'syntax_model1',
                        

['mean_model0',
 'argmax0',
 'cohesion_model0',
 'cohesion_abs_delta_model0',
 'syntax_model0',
 'syntax_abs_delta_model0',
 'vocabulary_model0',
 'vocabulary_abs_delta_model0',
 'phraseology_model0',
 'phraseology_abs_delta_model0',
 'grammar_model0',
 'grammar_abs_delta_model0',
 'conventions_model0',
 'conventions_abs_delta_model0',
 'mean_model1',
 'argmax1',
 'cohesion_model1',
 'cohesion_abs_delta_model1',
 'syntax_model1',
 'syntax_abs_delta_model1',
 'vocabulary_model1',
 'vocabulary_abs_delta_model1',
 'phraseology_model1',
 'phraseology_abs_delta_model1',
 'grammar_model1',
 'grammar_abs_delta_model1',
 'conventions_model1',
 'conventions_abs_delta_model1']

In [15]:
LABEL_COL = 'cohesion'
LABEL_COL = 'vocabulary'
LABEL_COL = 'grammar'

In [16]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [17]:
X_train.head()

,mean_model0,argmax0,cohesion_model0,cohesion_abs_delta_model0,syntax_model0,syntax_abs_delta_model0,vocabulary_model0,vocabulary_abs_delta_model0,phraseology_model0,phraseology_abs_delta_model0,...,syntax_model1,syntax_abs_delta_model1,vocabulary_model1,vocabulary_abs_delta_model1,phraseology_model1,phraseology_abs_delta_model1,grammar_model1,grammar_abs_delta_model1,conventions_model1,conventions_abs_delta_model1
782,3.056641,5,2.929688,0.041534,2.927734,0.042175,3.189453,0.043457,3.214844,0.051758,...,2.980469,0.044464,3.218750,0.031921,3.207031,0.028183,3.328125,0.067017,2.951172,0.053864
783,2.523438,2,2.423828,0.039459,2.417969,0.041809,2.769531,0.097534,2.630859,0.042572,...,2.449219,0.045654,2.847656,0.109619,2.597656,0.012177,2.550781,0.006088,2.371094,0.076111
784,2.628906,5,2.867188,0.090637,2.615234,0.005199,2.849609,0.083923,2.593750,0.013374,...,2.656250,0.003691,2.888672,0.091492,2.621094,0.009598,2.525391,0.045746,2.423828,0.084106
785,3.046875,5,3.125000,0.025635,2.955078,0.030121,3.093750,0.015381,3.152344,0.034607,...,2.964844,0.026306,3.146484,0.033356,3.113281,0.022446,3.273438,0.075073,2.771484,0.089783
786,3.550781,0,3.398438,0.042908,3.476562,0.020905,3.703125,0.042908,3.675781,0.035217,...,3.507812,0.029709,3.746094,0.036194,3.699219,0.023224,3.613281,0.000540,3.583984,0.008644


In [18]:
X_valid

,mean_model0,argmax0,cohesion_model0,cohesion_abs_delta_model0,syntax_model0,syntax_abs_delta_model0,vocabulary_model0,vocabulary_abs_delta_model0,phraseology_model0,phraseology_abs_delta_model0,...,syntax_model1,syntax_abs_delta_model1,vocabulary_model1,vocabulary_abs_delta_model1,phraseology_model1,phraseology_abs_delta_model1,grammar_model1,grammar_abs_delta_model1,conventions_model1,conventions_abs_delta_model1
0,2.648438,4,2.794922,0.055298,2.558594,0.033936,2.847656,0.075195,2.691406,0.016220,...,2.621094,0.020432,2.851562,0.065674,2.642578,0.012405,2.492188,0.068604,2.634766,0.015327
1,3.144531,1,3.093750,0.016144,3.011719,0.042236,3.191406,0.014908,3.128906,0.004971,...,3.082031,0.020477,3.193359,0.014900,3.164062,0.005585,3.113281,0.010551,3.230469,0.026688
2,3.626953,3,3.595703,0.008614,3.591797,0.009689,3.660156,0.009155,3.802734,0.048462,...,3.619141,0.015930,3.740234,0.016998,3.808594,0.035583,3.748047,0.019119,3.523438,0.041962
3,2.306641,2,2.265625,0.017776,2.173828,0.057587,2.667969,0.156616,2.326172,0.008469,...,2.373047,0.028778,2.744141,0.123108,2.365234,0.031982,2.310547,0.054352,2.408203,0.014389
4,2.560547,5,2.841797,0.109863,2.494141,0.025940,2.818359,0.100708,2.623047,0.024414,...,2.546875,0.028320,2.851562,0.087952,2.623047,0.000745,2.466797,0.058868,2.353516,0.102112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,2.958984,2,2.880859,0.026398,2.769531,0.064026,3.210938,0.085144,3.037109,0.026398,...,2.824219,0.043640,3.113281,0.054230,3.023438,0.023804,2.919922,0.011246,2.966797,0.004631
778,2.812500,2,2.978516,0.059021,2.685547,0.045135,3.054688,0.086121,2.875000,0.022217,...,2.718750,0.029968,3.023438,0.078735,2.794922,0.002787,2.759766,0.015327,2.648438,0.055054
779,2.492188,2,2.656250,0.065857,2.347656,0.057983,2.718750,0.090881,2.525391,0.013321,...,2.392578,0.041473,2.738281,0.097046,2.544922,0.019562,2.445312,0.020340,2.285156,0.084534
780,4.164062,0,3.931641,0.055817,4.066406,0.023453,4.238281,0.017822,4.179688,0.003752,...,4.132812,0.012138,4.226562,0.010269,4.179688,0.000934,4.304688,0.028946,4.304688,0.028946


In [19]:
preds = []
for row in X_valid.itertuples():
  if row.argmax0 == 4:
    preds.append(row.grammar_model0)
  else:
    preds.append((row.grammar_model1 + row.grammar_model0)/2)

In [20]:
calc_metric(dvalid[LABEL_COL].values, preds)

0.4479939341490076

In [21]:
calc_metric(dvalid[LABEL_COL].values, X_valid['grammar_model0'].values)

0.44891331292337533

In [22]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression() # Creating an Instance of LinearRegression model

lm.fit(X_train,y_train) # Train/fit on the trainingdata, this will give-

LinearRegression()

In [23]:
lm.coef_

array([-1.12890730e+01,  9.36552622e-03,  1.79271700e+00,  4.45591486e-01,
        2.31267506e+00,  1.08835531e+00,  2.19362649e+00, -8.30462172e-01,
        1.60659630e+00, -2.07060724e-01,  2.24797132e+00,  5.16395930e-01,
        1.55670627e+00, -3.20653767e-01,  2.35220067e+01,  3.30801292e-03,
       -3.90263416e+00, -8.11445943e-01, -3.41107564e+00,  2.96734097e+00,
       -4.59654653e+00,  1.10591031e+00, -3.68333904e+00, -6.41449146e-01,
       -3.53736535e+00, -6.66660611e-01, -3.74792796e+00,  3.66497855e-01])

In [24]:
pred = lm.predict(X_valid)
pred

array([2.44864506, 3.18450184, 3.72396676, 2.1871441 , 2.46213344,
       3.76532604, 2.04904857, 3.14234591, 1.98239374, 2.47055521,
       2.36608063, 2.26532857, 2.94534219, 3.48589551, 2.70137859,
       3.48671568, 2.06408054, 2.93989577, 3.03376178, 2.38508219,
       2.76204118, 3.3092701 , 3.76200617, 3.26690939, 3.49336454,
       2.19212868, 3.70397582, 3.86373922, 3.19935938, 3.1103217 ,
       2.53108607, 3.60817825, 3.23194897, 2.99389317, 3.61256544,
       2.60865487, 3.64551851, 3.31749964, 2.62187733, 3.41028421,
       2.89754672, 3.49924556, 3.24608379, 2.91073662, 2.56237083,
       3.03850457, 2.43534559, 2.82408919, 2.89217588, 3.27348816,
       3.13875817, 2.18120661, 2.96891818, 3.83494229, 3.56764928,
       2.55346916, 3.264044  , 2.28784569, 3.03550665, 4.47682698,
       2.06735402, 3.67495042, 3.98827974, 2.47970259, 2.37459576,
       2.92883752, 3.28674251, 2.95110165, 2.03537308, 2.69182346,
       2.94112731, 3.05388127, 3.01819718, 2.69127454, 3.76266

In [25]:
calc_metric(dvalid[LABEL_COL].values, pred)

0.4473863171846146

In [26]:
def get_preds(idx, label_col):
  preds = []
  for row in X_valid.itertuples():
    row = row._asdict()
    # if row['argmax0'] == idx:
    #   preds.append(row[f'{label_col}_model1'])
    # else:
    preds.append((row[f'{label_col}_model0']+ row[f'{label_col}_model1'])/2)
  return preds

In [27]:
from sklearn.linear_model import Ridge

In [31]:
FOLDS = 5
scores = []
gezi.try_mkdir(f'../working/boost')
preds = [[] for _ in range(len(TARGETS))]
df = oof
for i, label_col in tqdm(enumerate(TARGETS)):
  for fold in tqdm(range(FOLDS)):
    dvalid = df[df.fold==fold]
    dtrain = df[df.fold!=fold]
    cols_ = [x for x in cols if label_col in x]
    # cols_ = cols
    X_train = dtrain[cols_]
    y_train = dtrain[label_col]
    X_valid = dvalid[cols_]
    y_valid = dvalid[label_col]
    lm = Ridge() # Creating an Instance of LinearRegression model
    lm.fit(X_train,y_train) 
    # model.save_model(f'../working/boost/{label_col}-{fold}.lr')
    dvalid[f'{label_col}_pred'] = lm.predict(X_valid)
    dvalid[f'{label_col}_pred'] = get_preds(i, label_col)
    preds[i].extend(list(dvalid[f'{label_col}_pred']))
    score = calc_metric(dvalid[LABEL_COL].values, dvalid[f'{label_col}_pred'])
    scores.append(score)
    # ic(
    #   label_col,
    #   fold, 
    #   score,
    #   np.asarray(scores).mean(),
    #   )

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
preds = np.asarray(preds).transpose(1,0)

In [33]:
calc_metrics(np.vstack(oof.label.values), preds)

OrderedDict([('score', 0.4494906238092544),
             ('score/cohesion', 0.4777962713199574),
             ('score/syntax', 0.4447975782138983),
             ('score/vocabulary', 0.4151463077023143),
             ('score/phraseology', 0.4541350693754741),
             ('score/grammar', 0.4648181989594171),
             ('score/conventions', 0.4402503172844653),
             ('score/cls', 0.47199219315051305),
             ('acc', 0.42090684394443023),
             ('acc/cohesion', 0.3799539759652263),
             ('acc/syntax', 0.4167732037841984),
             ('acc/vocabulary', 0.46330861672206597),
             ('acc/phraseology', 0.4129378675530555),
             ('acc/grammar', 0.4160061365379698),
             ('acc/conventions', 0.43646126310406547),
             ('score/max', 0.574811342094556),
             ('acc/max', 0.27844541038097675),
             ('score/min', 0.3723484588462301),
             ('acc/min', 0.49143441575044744),
             ('score/max2', 0.448762073